# PID Controller

The `PIDController` class provides a standard implementation of a Proportional-Integral-Derivative controller. PID control is a ubiquitous feedback control mechanism used in industrial control systems and a wide variety of other applications requiring continuously modulated control. This notebook explains the components of the PID controller and demonstrates its effectiveness in controlling a simple simulated system.

## How PID Control Works

A PID controller continuously calculates an `error` value as the difference between a desired `setpoint` and a measured `process_variable`. It attempts to minimize the error over time by adjusting a `control_signal`.

- **Proportional (P) term:** The output is proportional to the current error. A high P gain results in a strong response to the error, but can lead to instability.
- **Integral (I) term:** This term accumulates past errors. It is designed to eliminate the steady-state error that can occur with a P-only controller.
- **Derivative (D) term:** This term responds to the rate of change of the error. It has a damping effect and can reduce overshoot and improve stability.

### Anti-Windup
This implementation includes an important feature called **anti-windup**. When the controller's output is saturated (at its maximum or minimum limit), the integral term can grow uncontrollably, a phenomenon known as 'windup'. This can cause significant overshoot when the system finally moves away from the limit. Our controller prevents this by stopping the integration when the output is saturated.

## Simulation Example

In this example, we will use the `PIDController` to manage the water level in a simple simulated tank. The tank has an inflow (controlled by the PID) and a constant outflow (a leak). The goal is to maintain the water level at a specific setpoint. We will also introduce a disturbance (a sudden increase in the leak) to see how the controller adapts.

**Note:** To run this notebook, ensure you have `matplotlib` and `numpy` installed (`pip install matplotlib numpy`) and that you are running the Jupyter server from the root directory of the project.

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
from swp.local_agents.control.pid_controller import PIDController
from swp.core.interfaces import State

# A simple simulation of a tank (our 'plant')
class SimpleTank:
    def __init__(self, initial_level, area, leak_rate):
        self.level = initial_level
        self.area = area
        self.leak_rate = leak_rate
        
    def update(self, inflow, dt):
        delta_level = (inflow - self.leak_rate) / self.area * dt
        self.level += delta_level
        return self.level

# --- Simulation Setup ---
setpoint = 10.0 # Target water level (m)
pid = PIDController(
    Kp=1.2, Ki=0.1, Kd=0.5,
    setpoint=setpoint,
    min_output=0.0,  # Inflow can't be negative
    max_output=10.0  # Max pump inflow rate
)

tank = SimpleTank(initial_level=0.0, area=100.0, leak_rate=2.0)

# --- Simulation Loop ---
dt = 1.0
duration = 200
history = {'time': [], 'level': [], 'control_signal': [], 'setpoint': []}

for t in range(int(duration/dt)):
    # Introduce a disturbance halfway through
    if t * dt > duration / 2:
        tank.leak_rate = 4.0
        
    # Controller computes action
    current_state = State(process_variable=tank.level)
    control_signal = pid.compute_control_action(current_state, dt)
    
    # Plant is updated
    tank.update(control_signal, dt)
    
    # Record history
    history['time'].append(t * dt)
    history['level'].append(tank.level)
    history['control_signal'].append(control_signal)
    history['setpoint'].append(setpoint)

# --- Plotting ---
fig, ax1 = plt.subplots(figsize=(12, 6))
ax1.plot(history['time'], history['level'], 'b-', label='Tank Level')
ax1.plot(history['time'], history['setpoint'], 'k--', label='Setpoint')
ax1.axvline(x=duration/2, color='r', linestyle='--', label='Disturbance (Increased Leak)')
ax1.set_xlabel('Time (s)')
ax1.set_ylabel('Water Level (m)', color='b')
ax1.tick_params(axis='y', labelcolor='b')
ax1.set_title('PID Control of a Water Tank')
ax1.legend(loc='lower right')
ax1.grid(True)

ax2 = ax1.twinx()
ax2.plot(history['time'], history['control_signal'], 'g:', label='Control Signal (Inflow)')
ax2.set_ylabel('Inflow (m^3/s)', color='g')
ax2.tick_params(axis='y', labelcolor='g')
ax2.legend(loc='upper right')

fig.tight_layout()
plt.show()
